# Start JupyterLab

To get the Jupyter kernel to work, run the following command in the terminal:

```bash
pixi run jupyter lab --no-browser
```
Then find the portion that reads:

```bash
Or copy and paste one of these URLs:
    http://localhost:8888/lab
    http://127.0.0.1:8888/lab
```

Copy and paste the URL into the spot that shows up at "Existing Jupyter Server" menu under "Select Kernel" button at the top right.

# Imports

In [69]:
from imgtools.io import read_dicom_series
from imgtools.autopipeline import AutoPipeline
import os
import sys
import pandas as pd
from readii.loaders import loadSegmentation
from readii.feature_extraction import singleRadiomicFeatureExtraction

# Define project root 
project_root = "C:/Users/azita/OneDrive/Documents/BHK lab/ispy-readii"

# Add the project root to sys.path if not already there
if project_root not in sys.path:
    sys.path.append(project_root)

from workflow.scripts.python.get_roi_names_with_count import getRTStructsRoiNames


# File path setup

This is an example with MRI data and RTSTRUCT

In [43]:
# Path to the DIRECTORY that contains all the CT dcm files
image_dir_path = os.path.join(project_root, "rawdata/Breast/ISPY2-100899/MR")
# Path to the RTSTRUCT FILE

rtstruct_file_path = os.path.join(project_root, "rawdata/Breast/ISPY2-100899/SEG/1-1.dcm")


# Run Med-Imagetools crawl

This creates a file used by READII to associate the images with the corresponding segmentation files.

In [66]:
# #Debugging 
# import SimpleITK as sitk
# from readii.loaders import loadSegmentation
# import os

# # Load segmentation file
# seg_image_dict = loadSegmentation(rtstruct_file_path, modality="SEG", baseImageDirPath=image_dir_path)
# print("Loaded ROI names:", seg_image_dict.keys())

# # SEG dimensions
# seg_mask = seg_image_dict['VOLSER Analysis Mask']
# print("Segmentation mask dimensions:", seg_mask.GetSize())

# # MRI image (first DICOM)
# mri_files = [f for f in os.listdir(image_dir_path) if f.endswith('.dcm')]
# image = sitk.ReadImage(os.path.join(image_dir_path, mri_files[0])) if mri_files else None
# print("MRI image dimensions:", image.GetSize() if image else "No DICOM files found")

# # TODO: segmentation mask has 80 slices, whereas the MRI image has only 51 slices?

Loaded ROI names: dict_keys(['VOLSER Analysis Mask'])
Segmentation mask dimensions: (256, 256, 80, 1)
MRI image dimensions: (256, 256, 1)


In [71]:
# Define input and output directories
mit_input_dir = os.path.join(project_root, "rawdata", "Breast")
mit_output_dir = os.path.join(project_root, "procdata", "Breast", "mit_output")

# Debug input and output directories
print("Input directory:", mit_input_dir)
print("Output directory:", mit_output_dir)

# Set up pipeline
modalities = "MR,SEG"
dry_run = True


# Initialize pipeline
pipeline = AutoPipeline(
    input_directory=mit_input_dir,
    output_directory=mit_output_dir,
    modalities=modalities,
    dry_run=dry_run,
    show_progress=True,
    verbose=True,
    update=True
    )


pipeline.run()


Input directory: C:/Users/azita/OneDrive/Documents/BHK lab/ispy-readii\rawdata\Breast
Output directory: C:/Users/azita/OneDrive/Documents/BHK lab/ispy-readii\procdata\Breast\mit_output


100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]


# List ROI names in the RTSTRUCT file

To load in the RTSTRUCT and get just the ROI we want, we need to make a dictionary with all the possible variations of the ROI names.

E.g. for a GTV ROI, we might have: GTV, GTVp, GTV1, GTP-p, etc.

Using this will rename any ROI that matches one of these names to GTV.

In [72]:
# List all the ROI names present in the RTSTRUCT 
#getRTStructsRoiNames(mit_input_dir)

{}

In [ ]:
# Select the name you want to give to your ROIs (e.g. GTV or tumor) and list any ROI names from previous cell output you want to include
# roi_name = "GTV"
# roi_name_options = {roi_name: ["Tumor_c40"]}

# Load in image and segmentation

In [ ]:
# Load the image (CT or MR)
image = read_dicom_series(image_dir_path)

# Load in the segmentation file (this is set up for RTSTRUCT)
# This will return a dictionary of segmentation images, with the keys being the ROI names
seg_image_dict = loadSegmentation(rtstruct_file_path, modality = 'SEG', 
                                  baseImageDirPath=image_dir_path)

# Get the segmentation image for the ROI of interest
seg_image = seg_image_dict['VOLSER Analysis Mask']



<function dict.keys>

# Confirm that loading working correctly

The dimensions of the image and segmenation should match.

In [79]:
arr_seg = sitk.GetArrayFromImage(seg_image)
np.unique(arr_seg)
# TODO: look at the the SEG from SLIcer --> multiple ressions, find tumor
# Multiple label values present in this segmentation. Must all be the same.

array([ 0,  1,  2, 17, 32, 33, 34, 49], dtype=uint8)

In [76]:
# Pick which config settings to use for PyRadiomics feature extraction
pyradiomics_param_file_path = os.path.join(project_root,"workflow/config/default_pyradiomics.yaml")

# Run PyRadiomics feature extraction with REAII function
radiomic_features_dict = singleRadiomicFeatureExtraction(image, seg_image, pyradiomics_param_file_path, randomSeed=10)

# Convert to dataframe
rad_features_df = pd.DataFrame.from_dict(radiomic_features_dict, orient="index").transpose()

ValueError: Multiple label values present in this segmentation. Must all be the same.

# Save out the results of the feature extraction

In [ ]:
save_radiomics_path = os.path.join(project_root, "results/4D-Lung/readii_radiomic_features.csv")

if not os.path.exists(os.path.dirname(save_radiomics_path)):
    os.makedirs(os.path.dirname(save_radiomics_path))

rad_features_df.to_csv(save_radiomics_path, index=False)